In [7]:
notebookName = 'audio-deepfake-detection-testing'
# runJobId = 'ASVspoof-2019_small-eval-1'
# runJobId = 'ASVspoof-2019_eval-1'
# runJobId = 'ASVspoof-2019_eval-1_large-batch'
runJobId = 'ASVspoof-2019_eval-1_huge-batch'
# runJobId = 'ASVspoof-2019_training'     # This should fail

import joblib
import numpy as np
from tensorflow.keras.utils import to_categorical

import configuration.configuration as configuration
from mel_spectrogram.mel_spectrogram import MelSpectrogramGenerator
from notebook_utils import notebookToPython
from processors.basic_model_evaluation_processor import BasicModelEvaluationProcessor
from readers.label_reader import readTrainingLabelsWithJob

In [8]:
config = configuration.ConfigLoader('config.yml')

notebookToPython(notebookName)
job = config.getJobConfig(runJobId)

import json
prettyJson = json.dumps(job.__dict__, indent=4)
print(f"job: {prettyJson}")

if (job.newModelGenerated):
    raise ValueError("This notebook is meant for testing. Select a job with a value for 'persisted-model' set.")

Write python file
Using configured model name: results/ASVspoof-2019_training_2025-03-23T13-49-37.076926.libjob
Assigned model name: results/ASVspoof-2019_training_2025-03-23T13-49-37.076926.libjob
job: {
    "jobId": "ASVspoof-2019_eval-1_huge-batch",
    "inputFileBatchSize": 71237,
    "outputFolder": "output",
    "dataPathRootRaw": "$HOMEDRIVE$HOMEPATH/workspace/Deepfake/data/ASVspoof-2019",
    "dataPathRoot": "C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019",
    "dataPathSuffix": "LA/ASVspoof2019_LA_eval/flac",
    "dataExtension": ".flac",
    "trainingSplitRandomState": 3,
    "labelFilename": "LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt",
    "executeToCategoricalForLabels": true,
    "numClasses": 2,
    "sampleRate": 16000,
    "duration": 5,
    "numMels": 128,
    "maxTimeSteps": 109,
    "optimizer": "adam",
    "loss": "categorical_crossentropy",
    "metrics": [
        "accuracy"
    ],
    "batchSize": 32,
    "numEpochs": 10,
    "newModelG

In [9]:
generator = MelSpectrogramGenerator()
model = joblib.load(job.persistedModel)
evaluationProc = BasicModelEvaluationProcessor(job, model)

In [10]:
fullDataPath = job.fullJoinFilePath(job.dataPathRoot, job.dataPathSuffix)
labels = readTrainingLabelsWithJob(job)

Loading C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt...


In [11]:
def processArrays(X, y):
    _X = np.array(X)
    _y = np.array(y)
    evaluationProc.process(_X, _y)


In [6]:
X = []
y = []

for filename, label in labels.items():
    _X, _y = generator.generateMelSpectrogram(job, fullDataPath, filename, label)
    X.append(_X)
    y.append(_y)

    if (len(X) >= job.inputFileBatchSize):
        processArrays(X, y)
        X = []
        y = []

if (len(X) > 0):
    processArrays(X, y)

print("\n")
report = evaluationProc.reportSnapshot()
evaluationProc.writeReportToFile(job.persistedModelResults, report)

2227/2227 ━━━━━━━━━━━━━━━━━━━━ 140s 62ms/step
  Batches: 1 - Files: 71237 - Score: 0.9006274829091624 - Elements: 71237


---- Testing (start) ----
start time: 2025-03-23T22:11:54.369844+00:00
end time: 2025-03-23T22:14:53.678899+00:00
elapsed: 0:02:59.309055

model file: results/ASVspoof-2019_training_2025-03-23T13-49-37.076926.libjob
batch count: 1
file count: 71237
accuracy_score: 0.9006274829091624

job: {
    "jobId": "ASVspoof-2019_eval-1_huge-batch",
    "inputFileBatchSize": 71237,
    "outputFolder": "output",
    "dataPathRootRaw": "$HOMEDRIVE$HOMEPATH/workspace/Deepfake/data/ASVspoof-2019",
    "dataPathRoot": "C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019",
    "dataPathSuffix": "LA/ASVspoof2019_LA_eval/flac",
    "dataExtension": ".flac",
    "trainingSplitRandomState": 3,
    "labelFilename": "LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt",
    "executeToCategoricalForLabels": true,
    "numClasses": 2,
    "sampleRate": 16000,
    "duration": 5,
